# Application

In [0]:
input_ts = '2017-07-18 12:00:00' #the time you want to do your trip- please note to use the same format

In [0]:
%sh
pip install geopy

In [0]:
%sh
pip install gdown

In [0]:
%sh
pip install folium

In [0]:
%sh
pip install pyshp

In [0]:
%sh
wget --output-file="logs.csv" "https://docs.google.com/spreadsheets/d/1bcdwwbj5N4l6gRwyb_7bC1CxvXj8uJxwy_6DFapEP8M/gviz/tq?tqx=out:csv&sheet=station_to_attraction_new" -O "station_to_attraction_new.csv"

In [0]:
%sh
wget --output-file="logs.csv" "https://docs.google.com/spreadsheets/d/1jfJ_nbPkeOhyFJg3FE9T4sextr3YI1sN6TY4InQCSUc/gviz/tq?tqx=out:csv&sheet=line_and_station_new" -O "line_and_station_new.csv"



In [0]:
%sh
wget --output-file="logs.csv" "https://docs.google.com/spreadsheets/d/1TzKC5vvs9AhppX8DDS25tH-MnCRtpkD3AAxJF8h4tMY/gviz/tq?tqx=out:csv&sheet=bus_stops" -O "bus_stops.csv"

In [0]:
%sh
wget --output-file="logs.csv" "https://docs.google.com/spreadsheets/d/18OgXI5HpIxPc8k0gKgTAOXeN2kl48MtNVyCFwZb_crw/gviz/tq?tqx=out:csv&sheet=bus_stops_in_each_journey_new" -O "bus_stops_in_each_journey_new.csv"

In [0]:
%sh
wget --output-file="logs.csv" "https://docs.google.com/spreadsheets/d/1IHqNxSeSq6yddHh_fQ7OCIS8y9pWtRoBLlvnEnrav70/gviz/tq?tqx=out:csv&sheet=attraction&station" -O "attraction&station.csv"

In [0]:
%sh
wget --output-file="logs.csv" "https://docs.google.com/spreadsheets/d/1NTmOn-w_6NRm9ShY-sgo5VS7Xx9kEBvBlvzoNmg1aas/gviz/tq?tqx=out:csv&sheet=attraction" -O "attraction.csv"

In [0]:
%sh
wget --output-file="logs.csv" "https://docs.google.com/spreadsheets/d/1NZ8hSdvuAWBnQQOza_KR-s7o2KhjjbrPKsogSn_u6pY/gviz/tq?tqx=out:csv&sheet=stop_to_stop" -O "stop_to_stop.csv"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, BooleanType, ArrayType, IntegerType, DateType, time
import pyspark.sql.functions as func
import pandas as pd
import re
from elasticsearch import Elasticsearch, helpers
import sys
import datetime
import numpy as np
import ast
import pickle
import os
import folium
import shapefile
from geopy import distance
from folium.features import DivIcon

In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("station", "14", 'Enter your current station')
station_to_atraction = pd.read_csv('station_to_attraction_new.csv')
station = dbutils.widgets.get("station")
attractions = station_to_atraction[station_to_atraction['station'] == int(station)]['attraction'].values
attractions = list(attractions)

In [0]:
dbutils.widgets.dropdown("atraction",attractions[0] , attractions,'Select your first attraction')


In [0]:
attraction_station = pd.read_csv('attraction&station.csv')

wanted_attraction1 = dbutils.widgets.get("atraction")
stations = ast.literal_eval(attraction_station[attraction_station['attraction'] == wanted_attraction1]['bus_stations'].values[0])
int_stations = list()
for idx, s in enumerate(stations):
    int_stations.append(int(s))
second_station = int_stations[0]
attractions = station_to_atraction[station_to_atraction['station'] ==second_station]['attraction'].values
attractions = list(attractions)
dbutils.widgets.dropdown("atraction2",attractions[0] , attractions,'Select your second attraction')
wanted_attraction2 = dbutils.widgets.get("atraction2")

In [0]:
stations = ast.literal_eval(attraction_station[attraction_station['attraction'] == wanted_attraction2]['bus_stations'].values[0])
int_stations = list()
for idx, s in enumerate(stations):
    int_stations.append(int(s))
third_station = int_stations[0]
attractions = station_to_atraction[station_to_atraction['station']==third_station]['attraction'].values
attractions = list(attractions)
dbutils.widgets.dropdown("atraction3",attractions[0] , attractions,'Select your third attraction')
wanted_attraction3 = dbutils.widgets.get("atraction3")

In [0]:
schema = StructType([
StructField('_id',StructType([StructField('$oid',StringType(),True)]),True),
StructField('atStop',BooleanType(),True),
StructField('busStop',LongType(),True),
StructField('congestion',BooleanType(),True),
StructField('delay',LongType(),True),
StructField('journeyPatternId',StringType(),True),
StructField('justLeftStop',BooleanType(),True),
StructField('justStopped',BooleanType(),True),
StructField('latitude',DoubleType(),True),
StructField('lineId',StringType(),True), 
StructField('longitude',DoubleType(),True),
StructField('timestamp',StructType([StructField('$numberLong',StringType(),True)]),True),
StructField('vehicleId',LongType(),True),
])

In [0]:
kafka_server = '10.0.0.30:9091'

kafka_raw_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_server) \
  .option("subscribePattern", "vehicleId_.*") \
  .option("startingOffsets", "earliest").option("maxOffsetsPerTrigger", 5000000).load()
 
kafka_value_df = kafka_raw_df.selectExpr("CAST(value AS STRING)")
 
schema = pickle.load(open("/dbfs/mnt/schema.pkl", "rb"))
 
kafka_df = kafka_value_df \
           .select(func.from_json(func.col("value"), schema=schema).alias('json')) \
           .select("json.*")
 

In [0]:
def clean_and_join_data(kafka_df):
  ################ CLEAN ####################
  bus_df = kafka_df.withColumn("_id", kafka_df["_id"]["$oid"].cast(StringType()))
  bus_df = bus_df.withColumnRenamed("_id", "id")

  #creat new columns
  bus_df = bus_df.withColumn("real_timestamp", bus_df["timestamp"]["$numberLong"].cast(LongType()) / 1000)
  bus_df = bus_df.withColumn("as_date", func.to_utc_timestamp(func.from_unixtime(bus_df.real_timestamp,'yyyy-MM-dd HH:mm:ss'),'GMT'))

  bus_df = bus_df.withColumn('timestamp', func.from_unixtime('real_timestamp'))
  bus_df = bus_df.withColumn('date',func.from_unixtime('real_timestamp').cast(DateType()))
  bus_df = bus_df.withColumn('hour', func.hour(bus_df.timestamp))
  bus_df = bus_df.withColumn('day_of_week', func.dayofweek(bus_df.timestamp))
  bus_df = bus_df.withColumn('month', func.month(bus_df.date))
  bus_df = bus_df.withColumn('year', func.year(bus_df.date))
  return bus_df

In [0]:
cleaned_df = clean_and_join_data(kafka_df)
print(station)
print(wanted_attraction1)

In [0]:
wanted_time = datetime.datetime.strptime(input_ts, '%Y-%m-%d %H:%M:%S')
before_time = wanted_time - datetime.timedelta(hours=1)
before_time = before_time.strftime('%Y-%m-%d %H:%M:%S')
input_line =station_to_atraction[(station_to_atraction['station']==int(station)) & (station_to_atraction['attraction']==wanted_attraction1)]['lines'].values[0]
input_line = ast.literal_eval(input_line)[0]

bus_stops_in_each_journey = pd.read_csv('line_and_station_new.csv')
line_route = bus_stops_in_each_journey[bus_stops_in_each_journey['lineId'] == input_line + "'"]['stops'].values[0]
stations = ast.literal_eval(line_route)

dest_s =station_to_atraction[(station_to_atraction['station']==int(station)) & (station_to_atraction['attraction']==wanted_attraction1)]['throw_station'].values[0]
dest_s = ast.literal_eval(dest_s)[input_line]
dest_s_index = stations.index(dest_s)

In [0]:
time_per_interval = dict()
optional_lines_and_s = dict()
init_ts = '2017-07-03 00:00:00' #the time our coustmer decide
init_ts = datetime.datetime.strptime(init_ts, '%Y-%m-%d %H:%M:%S')

initial_idx = stations.index(int(station))
final_idx = stations.index(dest_s)
stop_to_stop = pd.read_csv('stop_to_stop.csv') # ['origen_stop','dest_stop','lines']
initial_idx = stations.index(int(station))
final_idx = stations.index(dest_s)
wanted_stops = stations[initial_idx:final_idx+1]
optional_lines = list()
for idx in range(len(wanted_stops)):
  if idx == len(wanted_stops)-1:
    break
  time_per_interval[(wanted_stops[idx],wanted_stops[idx+1])] = [0,init_ts]
  wanted_patterns = stop_to_stop[(stop_to_stop['origen_stop'] ==wanted_stops[idx]) & (stop_to_stop['dest_stop'] ==wanted_stops[idx+1])]['lines'].values[0]
  wanted_patterns = ast.literal_eval(wanted_patterns)
  for idx, item in enumerate(wanted_patterns):
      line = ''
      for idx2, item2 in enumerate(item):
          if (idx2 < 2 and item2 == '0') or idx2 == 4:
              continue
          if idx2 == 2 and item2 == '0' and item[idx2 - 1] == '0':
              continue
          line += item2
      if line not in optional_lines:
        optional_lines.append(line)

In [0]:
limit_df = cleaned_df.select(cleaned_df.busStop,cleaned_df.timestamp, cleaned_df.journeyPatternId,cleaned_df.vehicleId,cleaned_df.justStopped, cleaned_df.atStop).filter(cleaned_df.timestamp < input_ts).filter(cleaned_df.lineId.isin(optional_lines))

In [0]:
esURL = "10.0.0.12:9200"
time_per_interval_df = pd.DataFrame(columns=['bus_stops','record_timestamp','travel_time'])

In [0]:
def foreach_batch_function_2(df, epoch_id):
  journies = df.select("journeyPatternId","vehicleId").distinct().rdd.flatMap(list).collect() 
#   v_ids = df.select("vehicleId").distinct().rdd.flatMap(list).collect() 
  j_list = [journies[i] for i in range(len(journies)) if i%2==0]
  v_list =  [journies[i] for i in range(len(journies)) if i%2!=0]
  for journey, v_id in zip(j_list,v_list):
    journy_bus_stops = df.select("busStop","timestamp").distinct().filter(df.journeyPatternId == journey).filter(df.vehicleId == v_id).filter((df.atStop == True) | (df.justStopped == True)).groupBy(df.busStop).agg({"timestamp": "max"}).orderBy("max(timestamp)")

    bus_stops = journy_bus_stops.select("busStop").rdd.flatMap(list).collect()

    optional_lines_and_s[(journey,v_id)] = bus_stops

    for idx in range(len(bus_stops)):
      if idx == len(bus_stops) -1:
        break
      first_b = bus_stops[idx]
      second_b = bus_stops[idx+1]
      if (first_b,second_b) in time_per_interval:
        timestamp_b = df.select("timestamp").filter(df.journeyPatternId == journey).filter(df.busStop == second_b).filter(df.vehicleId == v_id).filter((df.atStop == True) | (df.justStopped == True)).agg({"timestamp": "max"}).rdd.flatMap(list).collect()
        timestamp_b = datetime.datetime.strptime(timestamp_b[0], '%Y-%m-%d %H:%M:%S')
        if timestamp_b > time_per_interval[(first_b,second_b)][1]:
          timestamp_a = df.select("timestamp").filter(df.journeyPatternId == journey).filter(df.busStop == first_b).filter(df.vehicleId == v_id).filter((df.atStop == True) | (df.justStopped == True)).agg({"timestamp": "max"}).rdd.flatMap(list).collect()
          timestamp_a = datetime.datetime.strptime(timestamp_a[0], '%Y-%m-%d %H:%M:%S')
          time_per_interval[(first_b,second_b)] = (timestamp_b-timestamp_a, timestamp_b)
          # write to elastic
          d = {'bus_stops':(first_b,second_b),'record_timestamp':timestamp_b,'travel_time':timestamp_b-timestamp_a}
          tmp_df = pd.DataFrame(data=d) 
          tmp_df.write\
            .format("org.elasticsearch.spark.sql")\
            .option("es.nodes.wan.only","true")\
            .option("es.port","9200")\
            .option("es.nodes", esURL)\
            .mode("append")\
            .save("time_per_stops_stream")

In [0]:
limit_df.writeStream\
.format("memory") \
.foreachBatch(foreach_batch_function_2).start()

In [0]:
time_per_interval

In [0]:
changed = True
init_ts = '2017-07-03 00:00:00' #the time our coustmer decide
init_ts = datetime.datetime.strptime(init_ts, '%Y-%m-%d %H:%M:%S')
while changed:
  changed = False
  for kay,value in time_per_interval.items():
    if value[1] == init_ts:
      changed = True 

In [0]:
time_per_interval_df = pd.DataFrame(columns=['bus_stops','record_timestamp','travel_time'])
for key,value in time_per_interval.items():
  time_per_interval_df.at[-1] = [key,value[1],value[0]]
  time_per_interval_df.index = time_per_interval_df.index + 1
time_per_interval_df
time_per_interval_df

In [0]:
bus_stops_loc = pd.read_csv('bus_stops.csv')
dest_loc = (bus_stops_loc[bus_stops_loc['stop_code'] == dest_s]['Y'].values[0],bus_stops_loc[bus_stops_loc['stop_code'] == dest_s]['X'].values[0])
time = np.sum([value[0].seconds for key,value in time_per_interval.items()])
if len(str(time%60)) == 1:
  sum_times = str(int(np.round(time/60))) + ':0' + str(time%60)
else:
  sum_times = str(int(np.round(time/60))) + ':' + str(time%60)
sum_times

In [0]:
def find_bus_route(bus_line,x_point,y_point):
    shape = shapefile.Reader("NTA_Public_Transport.shp")
    shape_dict = {r.record.route_name: r.shape for r in shape.shapeRecords()}  # {lineid : Shape}
    if bus_line in shape_dict:
        parts = shape_dict[bus_line].parts
        points = shape_dict[bus_line].points
        geom = []
        for i in range(len(parts)):
            xy = []
            if i < len(parts) - 1:
                pt = points[parts[i]:parts[i + 1]]
            else:
                pt = points[parts[i]:]
            for x, y in pt:
                xy.append([y, x])
            geom.append(xy)
        return update_route(x_point, y_point, geom)


def update_route(x,y,points): #x=(lat,long), y=(lat,long), points=[[(lat,long),(lat,long),..][..]]
    closest_to_x, closest_to_y = None,None
    min_dist_to_x, min_dist_to_y = None,None
    for row_idx, row in enumerate(points):
        for p_idx, p in enumerate(row):
            dist_x = distance.distance(x, p).m
            dist_y = distance.distance(y, p).m
            if not closest_to_x:
                closest_to_x, closest_to_y = (row_idx, p_idx), (row_idx, p_idx)
                min_dist_to_x, min_dist_to_y = dist_x, dist_y
            if min_dist_to_x > dist_x:
                closest_to_x = (row_idx, p_idx)
                min_dist_to_x = dist_x
            if min_dist_to_y > dist_y:
                closest_to_y = (row_idx, p_idx)
                min_dist_to_y = dist_y
    if closest_to_x == closest_to_y:
        closest_to_y = (closest_to_y[0],closest_to_y[1]+1)
    if closest_to_x[0] <= closest_to_y[0]:
        new_route = points[closest_to_x[0]:closest_to_y[0] + 1]
        new_route[0] = new_route[0][closest_to_x[1] - 1:]
        new_route[-1] = new_route[-1][:closest_to_y[1] + 1]
    else:
        new_route = points[closest_to_y[0]:closest_to_x[0] + 1]
        new_route[0] = new_route[0][closest_to_y[1] - 1:]
        new_route[-1] = new_route[-1][:closest_to_x[1] + 1]
    return new_route


In [0]:
bus_stops_loc = pd.read_csv('bus_stops.csv')
att = pd.read_csv('attraction.csv')
station_to_attraction = station_to_atraction

os.system('gdown https://drive.google.com/uc?id=1wYA3aeOobJlaMhn4lYkoxOB5onaXUhxO')
os.system('gdown https://drive.google.com/uc?id=1eEJ7rk0Maq5W-F9lRnnKjC_kVN-_XqYr')
os.system('gdown https://drive.google.com/uc?id=1WPxk6MO1v4wnSWePVumeIAizCUwX-MZI')
os.system('gdown https://drive.google.com/uc?id=1VlB9aky1L0_p1cREEk4MR5KdraL6Bo5U')
os.system('gdown https://drive.google.com/uc?id=1IzD8Z1WTZ7GQL6RK14e3z3Rwdm83JLyo')

s1_lat = bus_stops_loc[bus_stops_loc['stop_code'] == int(station)]['Y'].values[0]
s1_long = bus_stops_loc[bus_stops_loc['stop_code'] == int(station)]['X'].values[0]
s2_lat = bus_stops_loc[bus_stops_loc['stop_code'] == second_station]['Y'].values[0]
s2_long = bus_stops_loc[bus_stops_loc['stop_code'] == second_station]['X'].values[0]
s3_lat = bus_stops_loc[bus_stops_loc['stop_code'] == third_station]['Y'].values[0]
s3_long = bus_stops_loc[bus_stops_loc['stop_code'] == third_station]['X'].values[0]

at1_lat = att[att['Name'] == wanted_attraction1]['Latitude'].values[0]
at1_long = att[att['Name'] == wanted_attraction1]['Longitude'].values[0]
at2_lat = att[att['Name'] == wanted_attraction2]['Latitude'].values[0]
at2_long = att[att['Name'] == wanted_attraction2]['Longitude'].values[0]
at3_lat = att[att['Name'] == wanted_attraction3]['Latitude'].values[0]
at3_long = att[att['Name'] == wanted_attraction3]['Longitude'].values[0]


d = {'long': [at1_long, at2_long, at3_long], 'lat': [at1_lat, at2_lat, at3_lat],
     'name': ['Attraction 1', 'Attraction 2', 'Attraction 3']}
df_loc = pd.DataFrame(data=d)

map_osm = folium.Map(location=[df_loc.iloc[0]["lat"], df_loc.iloc[0]["long"]], zoom_start=12)


folium.Marker(location=[s1_lat, s1_long],
              icon=folium.Icon(icon='bus', prefix='fa',color='red')).add_to(
    map_osm)

folium.Marker(location=[s2_lat, s2_long],
              icon=folium.Icon(icon='bus', prefix='fa',color='purple'),tooltip=f'station to {wanted_attraction2}').add_to(
    map_osm)
folium.Marker(location=[s3_lat, s3_long],
              icon=folium.Icon(icon='bus', prefix='fa',color='purple'),tooltip=f'station to {wanted_attraction3}').add_to(
    map_osm)

df_loc.apply(
    lambda row: folium.Marker(location=[row["lat"], row["long"]],
                              icon=folium.Icon(color='blue', icon='info-sign')).add_to(
        map_osm),
    axis=1)


## add bus line
possible_colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880',
                   '#FF97FF', '#FECB52']
first_lines = station_to_attraction[(station_to_attraction['attraction'] == wanted_attraction1) & (
        station_to_attraction['station'] == int(station))]['lines'].values[0]
first_lines = ast.literal_eval(first_lines)

for idx, item in enumerate(first_lines):
    item = item[:-1].upper()
    points = find_bus_route(item,(s1_lat,s1_long),(at1_lat,at1_long))
    if points:
        if len(possible_colors) == 0:
            color = 'blue'
        else:
            color = possible_colors[0]
            del possible_colors[0]
        folium.vector_layers.PolyLine(points, color=color,tooltip=item).add_to(map_osm)
try:
  second_lines = station_to_attraction[(station_to_attraction['attraction'] == wanted_attraction2) & (
          station_to_attraction['station'] == second_station)]['lines'].values[0]
except:
  pass
  
second_lines = ast.literal_eval(second_lines)
for idx, item in enumerate(second_lines):
    item = item[:-1].upper()
    points = find_bus_route(item, (s2_lat, s2_long), (at2_lat, at2_long))
    if points:
        if len(possible_colors) == 0:
            color = 'blue'
        else:
            color = possible_colors[0]
            del possible_colors[0]
        folium.vector_layers.PolyLine(points, color=color,tooltip=item).add_to(map_osm)

try:
  third_lines = station_to_attraction[(station_to_attraction['attraction'] == wanted_attraction3) & (
          station_to_attraction['station'] == third_station)]['lines'].values[0]
except:
    pass
    
third_lines = ast.literal_eval(third_lines)
for idx, item in enumerate(third_lines):
    item = item[:-1].upper()
    points = find_bus_route(item, (s3_lat, s3_long), (at3_lat, at3_long))
    if points:
        if len(possible_colors) == 0:
            color = 'blue'
        else:
            color = possible_colors[0]
            del possible_colors[0]
        folium.vector_layers.PolyLine(points, color=color,tooltip=item).add_to(map_osm)

folium.Marker((s2_lat, s2_long + 0.001), icon=DivIcon(
    icon_size=(150, 36),
    icon_anchor=(7, 20),
    html='<div style="font-size: 11pt; color : black; font-weight:bold; background:white; opacity:0.75;border:2px solid black;">station to attraction 2</div>',
)).add_to(map_osm)

folium.Marker((s3_lat, s3_long + 0.001), icon=DivIcon(
    icon_size=(150, 36),
    icon_anchor=(7, 20),
    html='<div style="font-size: 11pt; color : black; font-weight:bold; background:white; opacity:0.75;border:2px solid black;">station to attraction 3</div>',
)).add_to(map_osm)

folium.Marker((s1_lat, s1_long + 0.001), icon=DivIcon(
    icon_size=(150, 36),
    icon_anchor=(7, 20),
    html=f'<div style="font-size: 11pt; color : red; font-weight:bold; background:white; opacity:0.75;border:2px solid black;">you are here</div>',
)).add_to(map_osm)
folium.Marker((at1_lat, at1_long + 0.001), icon=DivIcon(
    icon_size=(150, 36),
    icon_anchor=(7, 20),
    html=f'<div style="font-size: 11pt; color : black;font-weight:bold;background:white;opacity:0.75;border:2px solid black;"> 1 {wanted_attraction1}</div>',
)).add_to(map_osm)
folium.Marker((at2_lat, at2_long + 0.001), icon=DivIcon(
    icon_size=(150, 36),
    icon_anchor=(7, 20),
    html=f'<div style="font-size: 11pt; color : black;font-weight:bold;background:white;opacity:0.75;border:2px solid black;"> 2 {wanted_attraction2}</div>',
)).add_to(map_osm)
folium.Marker((at3_lat, at3_long + 0.001), icon=DivIcon(
    icon_size=(150, 36),
    icon_anchor=(7, 20),
    html=f'<div style="font-size: 11pt; color : black;font-weight:bold;background:white;opacity:0.75;border:2px solid black;"> 3 {wanted_attraction3}</div>',
)).add_to(map_osm)

folium.Marker((s1_lat + 0.005, s1_long + 0.005), icon=DivIcon(
    icon_size=(150, 36),
    icon_anchor=(7, 20),
    html=f'<div style="font-size: 11pt; color : black;font-weight:bold;background:white;opacity:0.75;border:2px solid black;"> time estimated to arrive {wanted_attraction1} is {sum_times} minutes</div>',
)).add_to(map_osm)



In [0]:
os.system(f'git remote add origin https://shoval-z:<passward>@github.com/shoval-z/dublin.git')
os.system(f'git clone https://github.com/shoval-z/dublin.git')
os.system('git config --global user.email shovalzandberg1@gmail.com')
os.system('git config --global user.name shoval-z')

def commit_push(repo):
  os.chdir(f'/databricks/driver/dublin')
  os.system('git config --global push.default simple')
  os.system('git add index.html')
  os.system('git commit -m "update"')
  os.system(f'git push https://shoval-z:<passward>@github.com/shoval-z/dublin.git --all')
  
## In the place where you want to create the map ##
cwd = os.getcwd()
os.chdir('/databricks/driver/dublin')
map_osm.save('index.html')
commit_push('data')
os.chdir(cwd)

In [0]:
print(f'The needed lines for the first attractions are: {first_lines}')
url = att[att['Name'] == wanted_attraction1]['Url'].values[0]
phone = att[att['Name'] == wanted_attraction1]['Telephone'].values[0]
print(f'For more inforamtion you can contect {phone}\nand visit {url}\n')
print(f'The needed lines for the second attractions are: {second_lines}')
url = att[att['Name'] == wanted_attraction2]['Url'].values[0]
phone = att[att['Name'] == wanted_attraction2]['Telephone'].values[0]
print(f'For more inforamtion you can contect {phone}\nand visit {url}\n')
print(f'The needed lines for the third attractions are: {third_lines}')
url = att[att['Name'] == wanted_attraction3]['Url'].values[0]
phone = att[att['Name'] == wanted_attraction3]['Telephone'].values[0]
print(f'For more inforamtion you can contect {phone}\nand visit {url}\n')
print('To see the locations on map scroll down')

In [0]:
map_osm